In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [37]:
# pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import numpy as np
import pandas as pd

In [19]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4391719 sha256=46026e7264cf68dd011f0c71f97a9eff9954a225d1f7e4b482914c8babd5918f
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [21]:
df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)
df.sample(5)

<ipython-input-21-7fde084fdf7a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)


,text,class,char_length,token_length
176709,rapunzel cute hey hair long soft,0,32,6
48326,recommend tuber live gotten bored tuber curren...,0,456,60
217137,hama problem past relationship turn emotionall...,0,166,25
24644,seventh comment post gold award reply n't coun...,0,85,14
56108,got number girl class talked little bit asked ...,0,142,25


In [22]:
model = fasttext.load_model('drive/MyDrive/TextAnalytics/cc.en.300.bin')
def get_text_vector(sentence): return model.get_sentence_vector(sentence)

In [23]:
df['text_vectors'] = df['text'].apply(get_text_vector)
df.sample(5)

,text,class,char_length,token_length,text_vectors
158932,item leaving world sorry hurt sorry,1,44,9,"[-0.0050507784, 0.010136576, -0.0008624724, 0...."
67599,want people post faster browse post people sor...,0,78,12,"[0.02195679, 0.008976907, 0.020274734, 0.07857..."
229782,got head feeling good flipping coin decide dis...,0,65,11,"[-0.01273069, 0.009700653, 0.07175643, 0.09650..."
96469,love having migraine great time having migrain...,0,62,10,"[0.023815043, 0.0052314727, 0.03658324, 0.1003..."
16024,advice want end life going downhill,1,36,6,"[-0.0010206718, 0.033179864, 0.048775844, 0.09..."


In [24]:
X, y = df['text_vectors'], df['class']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [53]:
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
logreg_model_search = GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
logreg_model_search.fit(list(X_train), y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0]})

In [55]:
print(logreg_model_search.best_params_)
print(logreg_model_search.best_score_)

{'C': 10.0}
0.8952103309352039


In [57]:
logreg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=10, penalty='l2',max_iter=1000)
logreg_model = logreg.fit(list(X_train),y_train)

[LibLinear]

In [58]:
print(logreg_model.score(list(X_train), y_train))

0.8961894674614513


In [59]:
print(logreg_model.score(list(X_test), y_test))

0.8952110004741584


In [66]:
y_pred = logreg_model.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))

 Accuracy 0.8952
 Precision 0.8835 
 Recall 0.9119 
 F1 0.8974
